In [2]:
#Import ee and validate Google Earth Engine connection.
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=qR4FwQMtY95gbL8T7qLr14_4VWhY7t5EwMSOugZi_Lk&tc=4A9XoHtdC1ntcYvkne1SC6D9DLbg9jZtC7QiltD-r34&cc=eXfVXXObVWeQPGhBpVFw8NbaBg3nN-H0wyotbl00GVc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXlP0c0vmTX1Sd1aHXvtweozGUN6ETg9zgF_XTdzA062mYGgpo-XVa4

Successfully saved authorization token.


In [3]:
import geemap
import os
from geemap.conversion import *
import time

In [4]:
#Add vector file of area of interest - this is a shapefile around South Cubbington Wood
HS2 = ee.FeatureCollection('projects/ee-bripickstone/assets/HS2_cub')



#display the are of interest on open street map
Map = geemap.Map()
Map.centerObject(HS2)
Map.add('layer_manager')
Map.addLayer(HS2)
Map
#display(HS2)

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [12]:
#Import Surface reflectence Sentinel-2 data for the year 2021
#Filter this so only images are shown that overlap with HS2 area

S2_dataset_2021 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterDate('2021-01-01', '2021-12-31') \
    .filterBounds(HS2) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Select the first image in the collection
first_image_2021 = S2_dataset_2021.first()

# Print the acquisition date
acquisition_date = ee.Date(first_image_2021.get('system:time_start'))
print('Acquisition Date:', acquisition_date.format('YYYY-MM-dd').getInfo())

# Visualization parameters
visualization = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

# Create a median RGB composite
S2_median_RGB_composite = S2_dataset_2021.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'WVP']).median()

# Create a map using geemap
Map = geemap.Map()
Map.centerObject(HS2)
Map.add_layer_manager()  # This lets us switch between layers on the map
Map.addLayer(S2_median_RGB_composite, visualization, 'S2 RGB Median')
Map

#display(S2_dataset_2021.first())

Acquisition Date: 2021-02-28


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [13]:
# Specify the export region
export_region = HS2.geometry()

# Specify the export parameters
export_params = {
    'image': S2_median_RGB_composite,
    'description': 'S2_2021',
    'folder': 'HS2_2021',
    'scale': 10,
    'region': export_region,
}

# Export the image to Google Drive
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

# Monitor the task
while task.status()['state'] in ['READY', 'RUNNING']:
    print('Exporting...')
    time.sleep(5)

print('Export completed:', task.status())

Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Export completed: {'state': 'COMPLETED', 'description': 'S2_2021', 'creation_timestamp_ms': 1701956580379, 'update_timestamp_ms': 1701956860851, 'start_timestamp_ms': 1701956584349, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1mErSLKjI-vnZl5gYy

In [14]:
#Now complete the same for 2023

S2_dataset_2023 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterDate('2023-01-01', '2023-12-07') \
    .filterBounds(HS2) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Select the first image in the collection
first_image_2023 = S2_dataset_2023.first()

# Print the acquisition date
acquisition_date = ee.Date(first_image_2023.get('system:time_start'))
print('Acquisition Date:', acquisition_date.format('YYYY-MM-dd').getInfo())

# Visualization parameters
visualization = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

# Create a median RGB composite
S2_median_RGB_composite_2023 = S2_dataset_2023.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'WVP']).median()

# Create a map using geemap
Map = geemap.Map()
Map.centerObject(HS2)
Map.add_layer_manager()
Map.addLayer(S2_median_RGB_composite, visualization, 'S2 RGB Median')
Map

#display(S2_dataset_2023.first())

Acquisition Date: 2023-01-19


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [15]:
# Specify the export region
export_region = HS2.geometry()

# Specify the export parameters
export_params = {
    'image': S2_median_RGB_composite_2023,
    'description': 'S2_2023',
    'folder': 'HS2_2023',
    'scale': 10,
    'region': export_region,
}

# Export the image to Google Drive
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

# Monitor the task
while task.status()['state'] in ['READY', 'RUNNING']:
    print('Exporting...')
    time.sleep(5)

print('Export completed:', task.status())

Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Export completed: {'state': 'COMPLETED', 'descripti